# Nimbus Aspirate and Dispense Demo

This notebook demonstrates aspirate and dispense operations with the Hamilton Nimbus backend.

The demo covers:
1. Creating a Nimbus Deck and assigning resources
2. Setting up the NimbusBackend and LiquidHandler
3. Picking up tips from the tip rack
4. Aspirating 50 µL from wells (2mm above bottom)
5. Dispensing to wells (2mm above bottom)
6. Dropping tips to waste
7. Cleaning up and closing the connection


## Setup


In [ ]:
# Import necessary modules
import sys
import logging

from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends.hamilton.nimbus_backend import NimbusBackend
from pylabrobot.resources.hamilton.nimbus_decks import NimbusDeck
from pylabrobot.resources.hamilton.tip_racks import hamilton_96_tiprack_300uL_filter
from pylabrobot.resources.corning import Cor_96_wellplate_2mL_Vb
from pylabrobot.resources.coordinate import Coordinate

# Setup logging
plr_logger = logging.getLogger('pylabrobot')
plr_logger.setLevel(logging.INFO)  # INFO for normal use, DEBUG for troubleshooting
plr_logger.handlers.clear()
console_handler = logging.StreamHandler(sys.stdout)
console_handler.setFormatter(logging.Formatter('%(levelname)s - %(message)s'))
plr_logger.addHandler(console_handler)

# ========================================================================
# CREATE DECK AND RESOURCES (using coordinates from nimbus_deck_setup.ipynb)
# ========================================================================

# Create NimbusDeck using default values (layout 8 dimensions)
deck = NimbusDeck()

print(f"Deck created: {deck.name}")
print(f"  Size: {deck.get_size_x()} x {deck.get_size_y()} x {deck.get_size_z()} mm")
print(f"  Rails: {deck.num_rails}")

# Create and assign tip rack (HAM_FTR_300_0001)
# Using pre-calculated origin from nimbus_deck_setup.ipynb output:
# Tip rack origin (PyLabRobot): Coordinate(305.750, 126.537, 128.620)
tip_rack = hamilton_96_tiprack_300uL_filter(name="HAM_FTR_300_0001", with_tips=True)
deck.assign_child_resource(tip_rack, location=Coordinate(x=305.750, y=126.537, z=128.620))

print(f"\nTip rack assigned: {tip_rack.name}")

# Create and assign wellplate (Cor_96_wellplate_2mL_Vb_0001)
# Using pre-calculated origin from nimbus_deck_setup.ipynb output:
# Wellplate origin (PyLabRobot): Coordinate(438.070, 124.837, 101.490)
wellplate = Cor_96_wellplate_2mL_Vb(name="Cor_96_wellplate_2mL_Vb_0001", with_lid=False)
deck.assign_child_resource(wellplate, location=Coordinate(x=438.070, y=124.837, z=101.490))

print(f"Wellplate assigned: {wellplate.name}")
print(f"  Waste block: {deck.get_resource('default_long_block').name}")

# Serialize the deck #
#serialized = deck.serialize()
#with open("test_nimbus_deck.json", "w") as f:
#    json.dump(serialized, f, indent=2)

# Load from file and deserialize
#with open("test_nimbus_deck.json", "r") as f:
#   deck_data = json.load(f)
# Read deck from file example
# loaded_deck = NimbusDeck.deserialize(deck_data)

# Create NimbusBackend instance
# Replace with your instrument's IP address
backend = NimbusBackend(
    host="192.168.100.100",  # Replace with your instrument's IP
    port=2000,
    read_timeout=30,
    write_timeout=30
)

# Create LiquidHandler with backend and deck
lh = LiquidHandler(backend=backend, deck=deck)

print("LiquidHandler created successfully")

# Setup the robot
await lh.setup(unlock_door=False)

print("\n" + "="*60)
print("SETUP COMPLETE")
print("="*60)
print(f"Setup finished: {backend.setup_finished}")
print(f"\nInstrument Configuration:")
print(f"  Number of channels: {backend.num_channels}")


Deck created: deck
  Size: 831.85 x 424.18 x 300.0 mm
  Rails: 30

Tip rack assigned: HAM_FTR_300_0001
Wellplate assigned: Cor_96_wellplate_2mL_Vb_0001
  Waste block: default_long_block
LiquidHandler created successfully
INFO - Connecting to TCP server 192.168.100.100:2000...
INFO - Connected to TCP server 192.168.100.100:2000
INFO - Initializing Hamilton connection...
INFO - [INIT] Sending Protocol 7 initialization packet:
INFO - [INIT]   Length: 28 bytes
INFO - [INIT]   Hex: 1a 00 07 30 00 00 00 00 03 00 01 10 00 00 00 00 02 10 00 00 01 00 04 10 00 00 1e 00


INFO - [INIT] Received response:
INFO - [INIT]   Length: 28 bytes
INFO - [INIT]   Hex: 1a 00 07 30 00 00 00 00 03 00 01 11 00 00 02 00 02 11 07 00 01 00 04 11 00 00 1e 00
INFO - [INIT] ✓ Client ID: 2, Address: 2:2:65535
INFO - Registering Hamilton client...
INFO - [REGISTER] Sending registration packet:
INFO - [REGISTER]   Length: 48 bytes, Seq: 1
INFO - [REGISTER]   Hex: 2e 00 06 30 00 00 02 00 02 00 ff ff 00 00 00 00 fe ff 01 00 03 03 2a 00 00 00 00 00 00 00 00 00 00 00 02 00 02 00 ff ff 00 00 00 00 00 00 00 00
INFO - [REGISTER]   Src: 2:2:65535, Dst: 0:0:65534
INFO - [REGISTER] Received response:
INFO - [REGISTER]   Length: 48 bytes
INFO - [REGISTER] ✓ Registration complete
INFO - Discovering Hamilton root objects...
INFO - [DISCOVER_ROOT] Sending root object discovery:
INFO - [DISCOVER_ROOT]   Length: 52 bytes, Seq: 2
INFO - [DISCOVER_ROOT]   Hex: 32 00 06 30 00 00 02 00 02 00 ff ff 00 00 00 00 fe ff 02 00 03 13 2e 00 00 00 00 00 0c 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 0

## Define Resources

In [9]:
# Resources are already created in the setup cell above
# tip_rack and wellplate variables are available

print(f"Tip rack: {tip_rack.name} ({tip_rack.num_items} tips)")
print(f"Source/Destination plate: {wellplate.name} (using same plate, different wells)")

# Use wellplate as both source and destination
source_plate = wellplate
destination_plate = wellplate

# Get waste positions
waste_block = deck.get_resource("default_long_block")
waste_positions = waste_block.children[:4]

print(f"Waste positions: {[wp.name for wp in waste_positions]}")


Tip rack: HAM_FTR_300_0001 (96 tips)
Source/Destination plate: Cor_96_wellplate_2mL_Vb_0001 (using same plate, different wells)
Waste positions: ['default_long_1', 'default_long_2', 'default_long_3', 'default_long_4']


## Pick Up Tips

Pick up tips from positions A1-D1.


In [10]:
# Get the first 4 tip spots (A1, B1, C1, D1)
tip_spots = tip_rack["E4":"A5"]

print(f"Picking up tips from positions: {[ts.get_identifier() for ts in tip_spots]}")
await lh.pick_up_tips(tip_spots)

print("✓ Tips picked up successfully!")


Picking up tips from positions: ['E4', 'F4', 'G4', 'H4']
INFO - IsTipPresent parameters:
INFO - PickupTips parameters:
INFO -   tips_used: [1, 1, 1, 1]
INFO -   x_positions: [18844, 18844, 18844, 18844]
INFO -   y_positions: [-20499, -21399, -22299, -23199]
INFO -   traverse_height: 14600
INFO -   z_start_positions: [13802, 13802, 13802, 13802]
INFO -   z_stop_positions: [13002, 13002, 13002, 13002]
INFO -   tip_types: [<NimbusTipType.STANDARD_300UL_FILTER: 1>, <NimbusTipType.STANDARD_300UL_FILTER: 1>, <NimbusTipType.STANDARD_300UL_FILTER: 1>, <NimbusTipType.STANDARD_300UL_FILTER: 1>]
INFO -   num_channels: 4
INFO - PickupTips parameters:
INFO -   tips_used: [1, 1, 1, 1]
INFO -   x_positions: [18844, 18844, 18844, 18844]
INFO -   y_positions: [-20499, -21399, -22299, -23199]
INFO -   traverse_height: 14600
INFO -   z_start_positions: [13802, 13802, 13802, 13802]
INFO -   z_stop_positions: [13002, 13002, 13002, 13002]
INFO -   tip_types: [<NimbusTipType.STANDARD_300UL_FILTER: 1>, <Nimbu

## Aspirate Operation

Aspirate 50 µL from wells A1-D1, 2mm above the bottom of the well.


In [11]:
# Get source wells (A1, B1, C1, D1)
source_wells = source_plate["A7":"E7"]

print(f"Aspirating 50 µL from wells: {[w.get_identifier() for w in source_wells]}")
print(f"  Liquid height: 2.0 mm above bottom")

# Aspirate with liquid_height=2.0mm
# Tips are already picked up, so LiquidHandler will use them automatically
await lh.aspirate(
    source_wells,
    vols=[50.0, 50.0, 50.0, 50.0],  # Can be a single number (applies to all channels) or a list
    liquid_height=[2.0, 2.0, 2.0, 2.0],  # 2mm above bottom of well (can be a single float or list)
    flow_rates=[250.0, 250.0, 250.0, 250.0],
    liquid_seek_height=[5.0, 5.0, 5.0, 5.0],
)

print("✓ Aspiration complete!")


Aspirating 50 µL from wells: ['A7', 'B7', 'C7', 'D7']
  Liquid height: 2.0 mm above bottom
INFO - DisableADC parameters:
INFO -   tips_used: [1, 1, 1, 1]
INFO - Disabled ADC before aspirate
INFO - GetChannelConfiguration parameters:
INFO -   channel: 1
INFO -   indexes: [2]
INFO - GetChannelConfiguration parameters:
INFO -   channel: 2
INFO -   indexes: [2]
INFO - GetChannelConfiguration parameters:
INFO -   channel: 3
INFO -   indexes: [2]
INFO - GetChannelConfiguration parameters:
INFO -   channel: 4
INFO -   indexes: [2]
INFO - Aspirate parameters:
INFO -   aspirate_type: [0, 0, 0, 0]
INFO -   tips_used: [1, 1, 1, 1]
INFO -   x_positions: [35016, 35016, 35016, 35016]
INFO -   y_positions: [-16899, -17799, -18699, -19599]
INFO -   traverse_height: 14600
INFO -   liquid_seek_height: [500, 500, 500, 500]
INFO -   liquid_surface_height: [10594, 10594, 10594, 10594]
INFO -   submerge_depth: [0, 0, 0, 0]
INFO -   follow_depth: [0, 0, 0, 0]
INFO -   z_min_position: [10394, 10394, 10394, 10

## Dispense Operation

Dispense 50 µL to wells A2-D2, 2mm above the bottom of the well.


In [12]:
# Get destination wells (A2, B2, C2, D2)
dest_wells = destination_plate["A12":"E12"]

print(f"Dispensing 50 µL to wells: {[w.get_identifier() for w in dest_wells]}")
print(f"  Liquid height: 2.0 mm above bottom")

# Dispense with liquid_height=2.0mm
# Tips are already picked up, so LiquidHandler will use them automatically
await lh.dispense(
    dest_wells,
    vols=[50.0, 50.0, 50.0, 50.0],  # Can be a single number (applies to all channels) or a list
    liquid_height=[2.0, 2.0, 2.0, 2.0],  # 2mm above bottom of well (can be a single float or list)
    flow_rates=[400.0, 400.0, 400.0, 400.0],
    liquid_seek_height=[5.0, 5.0, 5.0, 5.0],
)

print("✓ Dispense complete!")


Dispensing 50 µL to wells: ['A12', 'B12', 'C12', 'D12']
  Liquid height: 2.0 mm above bottom
INFO - DisableADC parameters:
INFO -   tips_used: [1, 1, 1, 1]
INFO - Disabled ADC before dispense
INFO - GetChannelConfiguration parameters:
INFO -   channel: 1
INFO -   indexes: [2]
INFO - GetChannelConfiguration parameters:
INFO -   channel: 2
INFO -   indexes: [2]
INFO - GetChannelConfiguration parameters:
INFO -   channel: 3
INFO -   indexes: [2]
INFO - GetChannelConfiguration parameters:
INFO -   channel: 4
INFO -   indexes: [2]
INFO - Dispense parameters:
INFO -   dispense_type: [0, 0, 0, 0]
INFO -   tips_used: [1, 1, 1, 1]
INFO -   x_positions: [39516, 39516, 39516, 39516]
INFO -   y_positions: [-16899, -17799, -18699, -19599]
INFO -   traverse_height: 14600
INFO -   liquid_seek_height: [500, 500, 500, 500]
INFO -   dispense_height: [10594, 10594, 10594, 10594]
INFO -   submerge_depth: [0, 0, 0, 0]
INFO -   follow_depth: [0, 0, 0, 0]
INFO -   z_min_position: [10394, 10394, 10394, 10394]

## Drop Tips

Drop tips to waste positions.


In [13]:
print(f"Dropping tips at waste positions: {[wp.name for wp in waste_positions]}")
await lh.drop_tips(waste_positions)

print("✓ Tips dropped successfully!")


Dropping tips at waste positions: ['default_long_1', 'default_long_2', 'default_long_3', 'default_long_4']
INFO - DropTipsRoll parameters:
INFO -   tips_used: [1, 1, 1, 1]
INFO -   x_positions: [55375, 55375, 55375, 55375]
INFO -   y_positions: [1986, 188, -7615, -9413]
INFO -   traverse_height: 14600
INFO -   z_start_positions: [13539, 13539, 13539, 13539]
INFO -   z_stop_positions: [13139, 13139, 13139, 13139]
INFO -   z_final_positions: [14600, 14600, 14600, 14600]
INFO -   roll_distances: [900, 900, 900, 900]
INFO - Dropped tips on channels [0, 1, 2, 3]
✓ Tips dropped successfully!


## Cleanup

Finally, we'll stop the liquid handler and close the connection.


In [14]:
# Stop and close connection
await lh.backend.park()
await lh.backend.unlock_door()
await lh.stop()

print("Connection closed successfully")


INFO - Park parameters:
INFO - Instrument parked successfully
INFO - UnlockDoor parameters:
INFO - Door unlocked successfully


INFO - Hamilton backend stopped
Connection closed successfully
